In [55]:
import opendatasets as od

#Data analysis
import pandas as pd
import numpy as np

#Data visualization
import seaborn as sns
import matplotlib.pyplot as plt

#ML 
from sklearn.preprocessing import LabelEncoder

In [56]:
od.download("https://www.kaggle.com/datasets/ruchi798/data-science-job-salaries/data", force=True)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading data-science-job-salaries.zip to .\data-science-job-salaries


100%|██████████| 7.37k/7.37k [00:00<00:00, 1.26MB/s]

In [57]:
data = pd.read_csv("data-science-job-salaries\ds_salaries.csv")

In [58]:
data.head(5)

,Unnamed: 0,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L


In [59]:
data.columns.values

array(['Unnamed: 0', 'work_year', 'experience_level', 'employment_type',
       'job_title', 'salary', 'salary_currency', 'salary_in_usd',
       'employee_residence', 'remote_ratio', 'company_location',
       'company_size'], dtype=object)

In [60]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 607 entries, 0 to 606
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          607 non-null    int64 
 1   work_year           607 non-null    int64 
 2   experience_level    607 non-null    object
 3   employment_type     607 non-null    object
 4   job_title           607 non-null    object
 5   salary              607 non-null    int64 
 6   salary_currency     607 non-null    object
 7   salary_in_usd       607 non-null    int64 
 8   employee_residence  607 non-null    object
 9   remote_ratio        607 non-null    int64 
 10  company_location    607 non-null    object
 11  company_size        607 non-null    object
dtypes: int64(5), object(7)
memory usage: 57.0+ KB


In [61]:
data.describe()

,Unnamed: 0,work_year,salary,salary_in_usd,remote_ratio
count,607.000000,607.000000,6.070000e+02,607.000000,607.00000
mean,303.000000,2021.405272,3.240001e+05,112297.869852,70.92257
std,175.370085,0.692133,1.544357e+06,70957.259411,40.70913
min,0.000000,2020.000000,4.000000e+03,2859.000000,0.00000
25%,151.500000,2021.000000,7.000000e+04,62726.000000,50.00000
50%,303.000000,2022.000000,1.150000e+05,101570.000000,100.00000
75%,454.500000,2022.000000,1.650000e+05,150000.000000,100.00000
max,606.000000,2022.000000,3.040000e+07,600000.000000,100.00000


In [62]:
data.drop(['Unnamed: 0', 'salary', 'salary_currency'], axis=1, inplace=True)

In [63]:
data.head(5)

,work_year,experience_level,employment_type,job_title,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2020,MI,FT,Data Scientist,79833,DE,0,DE,L
1,2020,SE,FT,Machine Learning Scientist,260000,JP,0,JP,S
2,2020,SE,FT,Big Data Engineer,109024,GB,50,GB,M
3,2020,MI,FT,Product Data Analyst,20000,HN,0,HN,S
4,2020,SE,FT,Machine Learning Engineer,150000,US,50,US,L


In [66]:
encoder = LabelEncoder()

data['experience_level_encoded'] = encoder.fit_transform(data['experience_level'])
data['employment_type_encoded'] = encoder.fit_transform(data['employment_type'])
data['job_title_encoded'] = encoder.fit_transform(data['job_title'])
data['employee_residence_encoded'] = encoder.fit_transform(data['employee_residence'])
data['company_size_encoded'] = encoder.fit_transform(data['company_size'])
data['company_location_encoded'] = encoder.fit_transform(data['company_location'])

data.drop(['experience_level', 'employment_type', 'job_title', 'employee_residence', 'company_size', 'company_location'], axis=1, inplace=True)
data.head(5)


,work_year,salary_in_usd,remote_ratio,experience_level_encoded,employment_type_encoded,job_title_encoded,employee_residence_encoded,company_size_encoded,company_location_encoded
0,2020,79833,0,2,2,22,14,0,12
1,2020,260000,0,3,2,41,32,2,29
2,2020,109024,50,3,2,7,20,1,18
3,2020,20000,0,2,2,47,23,2,20
4,2020,150000,50,3,2,38,55,0,48


In [67]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

normalized_data = scaler.fit_transform(data)


normalized_df = pd.DataFrame(normalized_data, columns=data.columns)


normalized_df.head(5)


,work_year,salary_in_usd,remote_ratio,experience_level_encoded,employment_type_encoded,job_title_encoded,employee_residence_encoded,company_size_encoded,company_location_encoded
0,0.0,0.128904,0.0,0.666667,0.666667,0.448980,0.250000,0.0,0.244898
1,0.0,0.430620,0.0,1.000000,0.666667,0.836735,0.571429,1.0,0.591837
2,0.0,0.177789,0.5,1.000000,0.666667,0.142857,0.357143,0.5,0.367347
3,0.0,0.028705,0.0,0.666667,0.666667,0.959184,0.410714,1.0,0.408163
4,0.0,0.246409,0.5,1.000000,0.666667,0.775510,0.982143,0.0,0.979592


In [69]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(normalized_df, test_size=0.2, random_state=42)

train_data.head(5)


,work_year,salary_in_usd,remote_ratio,experience_level_encoded,employment_type_encoded,job_title_encoded,employee_residence_encoded,company_size_encoded,company_location_encoded
9,0.0,0.204543,0.5,1.000000,0.666667,0.673469,0.732143,1.0,0.775510
227,0.5,0.143676,0.5,0.666667,0.666667,0.448980,0.250000,0.0,0.244898
591,1.0,0.237791,1.0,1.000000,0.666667,0.326531,0.982143,0.5,0.979592
516,1.0,0.250596,1.0,1.000000,0.666667,0.428571,0.982143,0.5,0.979592
132,0.5,0.059519,1.0,0.666667,0.666667,0.081633,1.000000,0.5,0.979592


In [73]:
X_train = train_data.drop('salary_in_usd', axis=1)
Y_train = train_data['salary_in_usd']

X_val = val_data.drop('salary_in_usd', axis=1)
Y_val = val_data['salary_in_usd']

X_train.shape, Y_train.shape, X_val.shape, Y_val.shape

((485, 8), (485,), (122, 8), (122,))